In [47]:
import pandas as pd
import geopandas as gpd
import h3pandas
import numpy as np
import shapely
import json


In [3]:
"""
This cell is only for cleaning data, 
leave commented out unless you don't already have the clean data.
"""

# # load raw data from file
# cblocks = gpd.read_file('../data/raw/census_blocks.csv')

# # renames first column to correct geopandas format
# cblocks.columns = ['geometry', 'CB2020', 'BoroCode', 'BoroName', 'CT2020', 'BCTCB2020',
#        'GEOID', 'Shape_Leng', 'Shape_Area']
    
# # dictionary of each borocode
# boro_dict = {2:'bronx', 3:'brooklyn', 1:'manhattan', 4:'queens', 5:'staten'}

# # writes each borough to its own file
# for boro_code in range(1,6):
#     cblocks[cblocks["BoroCode"]== str(boro_code)].to_csv(f"../data/clean/census_blocks/{boro_dict[boro_code]}.csv",index=False)
    


"\nThis cell is only for cleaning data, \nleave commented out unless you don't already have the clean data.\n"

In [4]:
initial_rez = 13
parent_rez = 10

In [6]:
from shapely import wkt

df = pd.read_csv('../data/clean/census_blocks/all.csv',index_col=False)

df['geometry'] = df['geometry'].apply(wkt.loads)
gdf = gpd.GeoDataFrame(df, crs='epsg:4326')
# gdf.reset_index(drop=True)

# pd.read_csv('../data/clean/h3_index/brooklyn_hex.csv')
# exp_gdf = gdf.explode(index_parts=True).reset_index().drop('level_0',axis=1)

In [8]:
APERTURE_SIZE=initial_rez
# dfh = gdf.h3.polyfill_resample(APERTURE_SIZE,return_geometry=False)
dfh = gdf.h3.polyfill_resample(APERTURE_SIZE,return_geometry=True)


/home/thowe/anaconda3/envs/plz-gpu/lib/python3.8/site-packages/h3pandas/h3pandas.py:751: UserWarning: 8 rows did not generate a H3 cell.Consider using a finer resolution.
  warnings.warn(


In [9]:
op_dict = {'GEOID':pd.Series.nunique}

lout = dfh[['GEOID']].groupby(['h3_polyfill','GEOID'])

lout = lout.nunique()
lout = lout.reset_index()
lout = lout.set_index('h3_polyfill')



op_dict = {'GEOID':pd.Series.nunique}

lout['cell_count'] = 1
# op_dict = {'GEOID':pd.Series.nunique,'Shape_Area':pd.Series.first,'Shape_Leng':pd.Series.first,'h3_cell_area':'sum'}


# op_dict = {'Shape_Area':'sum','h3_cell_area':'sum'}
# op_dict = {'GEOID':pd.Series.nunique,'Shape_Area':'sum','h3_cell_area':'sum'}


In [18]:
op_dict = {'GEOID':pd.Series.mode,'h3_cell_area':'sum','cell_count':pd.Series.sum}
op_dict2 = {'GEOID':lambda x: list(set(list(x)))}



In [11]:
rout = lout.h3.cell_area()
final = rout.h3.h3_to_parent(parent_rez)

In [12]:
one_way = final.groupby(['h3_10']).agg(op_dict2)
another_a = final.groupby(['GEOID','h3_10']).agg(op_dict)

another = final.groupby(['GEOID']).agg(op_dict)
# final.drop('GEOID',axis=1).reset_index()

In [32]:
another_new = another.drop(['GEOID'],axis=1)
another_new = another_new.rename({'h3_cell_area':'GEOID_cell_area','cell_count':'GEOID_cell_count'}, axis=1)
another_new


,GEOID_cell_area,GEOID_cell_count
GEOID,,
360050001000002,0.001545,35
360050001001000,0.494692,11210
360050001001001,0.500402,11339
360050001001002,0.013504,306
360050001001003,0.675450,15305
...,...,...
360850323001023,0.003741,85
360850323001024,0.007173,163
360850323001025,0.105880,2406


In [14]:
another_x = another_a.drop(['GEOID'],axis=1)
another_x


another_x['proportion'] = another_x['cell_count'] / another_x.groupby('GEOID')['cell_count'].transform('sum')


In [69]:
another_x.rename({'h3_cell_area':'area_in_h3_cell','cell_count':'GEOID_cell_count','proportion':'proportion'}, axis=1)
another_x.reset_index().to_json('../data/clean/geoid_fragments.json',orient='index')
pd.read_json('../data/clean/geoid_fragments.json',orient='index').set_index(['GEOID','h3_10'])


h3_cell_area  cell_count  proportion
GEOID           h3_10                                                
360050001000002 8a2a100f408ffff      0.000839          19    0.542857
                8a2a100f40d7fff      0.000353           8    0.228571
                8a2a100f4707fff      0.000309           7    0.200000
                8a2a100f4727fff      0.000044           1    0.028571
360050001001000 8a2a100f0967fff      0.001191          27    0.002409
...                                       ...         ...         ...
360859901000006 8a2a1075e89ffff      0.003869          88    0.082243
                8a2a1075ec4ffff      0.001011          23    0.021495
                8a2a1075ec67fff      0.007782         177    0.165421
                8a2a1075ec6ffff      0.013233         301    0.281308
                8a2a1075ed4ffff      0.003429          78    0.072897

[194628 rows x 3 columns]

In [51]:
one_way = final.groupby(['h3_10']).agg(op_dict2)


In [52]:
h3_to_GEOID = one_way
h3_to_GEOID.to_json('../data/clean/h3_to_GEOID.json')

In [27]:
f = pd.read_json('../data/clean/h3_to_GEOID.json')

In [28]:
f

,GEOID
8a2a1000005ffff,[360050516021001]
8a2a10000087fff,[360050516015000]
8a2a10000097fff,[360050516015000]
8a2a1000009ffff,[360050516010001]
8a2a100000a7fff,[360050516015000]
...,...
8a2a10776d8ffff,[360470053012001]
8a2a10776d9ffff,[360470053012001]
8a2a10776da7fff,"[360470053012000, 360470053012001, 36047005301..."
8a2a10776daffff,"[360470053012001, 360470053012003]"
